In [ ]:
#动态录像，如果动则开始录r
import datetime
import cv2
import numpy as np
import time

sdThresh = 4 
font = cv2.FONT_HERSHEY_SIMPLEX

def distMap(frame1, frame2):
    frame1_32 = np.float32(frame1)
    frame2_32 = np.float32(frame2)
    diff32 = frame1_32 - frame2_32
    norm32 = np.sqrt(diff32[:,:,0]**2 + diff32[:,:,1]**2 + \
             diff32[:,:,2]**2)/np.sqrt(255**2 + 255**2 + 255**2)
    dist = np.uint8(norm32*255)
    return dist

def write(n):#录制时间段，秒
    _, frame1 = cap.read()
    _, frame2 = cap.read()
    facecount = 0
    while True:
        _, frame3 = cap.read()
        rows, cols, _ = np.shape(frame3)
        #cv2.imshow('dist', frame3)
        dist = distMap(frame1, frame3)

        frame1 = frame2
        frame2 = frame3

        mod = cv2.GaussianBlur(dist, (9,9), 0)

        _, thresh = cv2.threshold(mod, 100, 255, 0)
        _, stDev = cv2.meanStdDev(mod)

        #cv2.imshow('dist', mod)
        #cv2.putText(mod, "Standard Deviation - {}".format(round(stDev[0][0],0)), (70, 70), font, 1, (255, 0, 255), 1, cv2.LINE_AA)
        if stDev > sdThresh:
            now = str(datetime.datetime.now())[:19].replace(':', ':')
            cv2.putText(frame2, now, (5, 20), font, 0.5, (255, 255, 255), 1)
            aviFile.write(frame2)
            #print("Motion detected.. Do something!!!");
        cv2.imshow('frame', frame2)
        if cv2.waitKey(1) & 0xFF == 27:
            break
        if(time.time()-time1>n):
            break
cv2.namedWindow('frame')
#cv2.namedWindow('dist')

cap = cv2.VideoCapture(0)

#创建视频文件
name = 'video/'+str(datetime.datetime.now())[:19].replace(':', '_')+'.avi'
aviFile = cv2.VideoWriter(name,cv2.VideoWriter_fourcc('M','J','P','G'),25, (640,480))  #帧速和视频宽度、高度
time1=time.time()
write(100)#录制时间
aviFile.release()
cap.release()
cv2.destroyAllWindows()